In [18]:
import pandas as pd

## Load Dataset

In [19]:
df_fct = (
    pd.read_csv('data/fct_sales.csv')
    .assign(
        purchase_timestamp=lambda x: pd.to_datetime(x['purchase_timestamp'], format='%Y-%m-%d %H:%M:%S')
    )
)
df_fct.info()
display(df_fct.sample(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779495 entries, 0 to 779494
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   invoice_id          779495 non-null  int64         
 1   product_id          779495 non-null  object        
 2   customer_id         779495 non-null  int64         
 3   date_id             779495 non-null  object        
 4   order_amt           779495 non-null  int64         
 5   product_price       779495 non-null  float64       
 6   purchase_timestamp  779495 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 41.6+ MB


,invoice_id,product_id,customer_id,date_id,order_amt,product_price,purchase_timestamp
385144,536152,22644,16223,2010-11-30,4,1.45,2010-11-30 11:50:00
564666,559557,84877B,17444,2011-07-11,24,1.25,2011-07-11 10:33:00
768894,580611,22595,12748,2011-12-05,2,0.85,2011-12-05 11:49:00
210035,516590,85099C,15005,2010-07-21,2,1.95,2010-07-21 13:47:00
662787,570225,22402,16595,2011-10-09,1,0.39,2011-10-09 13:15:00
30768,493536,21805,17841,2010-01-04,10,0.42,2010-01-04 15:29:00
368891,534378,22666,14461,2010-11-22,2,2.95,2010-11-22 12:33:00
609060,564729,85204,13137,2011-08-28,4,0.12,2011-08-28 12:44:00
496075,550535,23204,14211,2011-04-19,10,0.85,2011-04-19 11:19:00
391264,536976,84029G,14723,2010-12-03,3,3.75,2010-12-03 14:18:00


In [20]:
df_customer = (
    pd.read_csv('data/dim_customer.csv')
)
df_customer.info()
display(df_customer.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5894 entries, 0 to 5893
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customer_id       5894 non-null   int64 
 1   customer_country  5894 non-null   object
dtypes: int64(1), object(1)
memory usage: 92.2+ KB


,customer_id,customer_country
0,13085,United Kingdom
1,13078,United Kingdom
2,15362,United Kingdom
3,18102,United Kingdom
4,12682,France


In [21]:
df_product = (
    pd.read_csv('data/dim_product.csv')
)
df_product.info()
display(df_product.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5315 entries, 0 to 5314
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_id           5315 non-null   object
 1   product_description  5315 non-null   object
dtypes: object(2)
memory usage: 83.2+ KB


,product_id,product_description
0,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS
1,79323P,PINK CHERRY LIGHTS
2,79323W,WHITE CHERRY LIGHTS
3,22041,"RECORD FRAME 7"" SINGLE SIZE"
4,21232,STRAWBERRY CERAMIC TRINKET BOX


In [22]:
df_date = (
    pd.read_csv('data/dim_date.csv')
)
df_date.info()
display(df_date.sample(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 739 entries, 0 to 738
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date_id      739 non-null    object
 1   year         739 non-null    int64 
 2   month        739 non-null    int64 
 3   day          739 non-null    int64 
 4   day_of_week  739 non-null    int64 
 5   is_weekend   739 non-null    bool  
 6   quarter      739 non-null    int64 
dtypes: bool(1), int64(5), object(1)
memory usage: 35.5+ KB


,date_id,year,month,day,day_of_week,is_weekend,quarter
566,2011-06-20,2011,6,20,0,False,2
25,2009-12-26,2009,12,26,5,True,4
212,2010-07-01,2010,7,1,3,False,3
378,2010-12-14,2010,12,14,1,False,4
222,2010-07-11,2010,7,11,6,True,3
438,2011-02-12,2011,2,12,5,True,1
177,2010-05-27,2010,5,27,3,False,2
530,2011-05-15,2011,5,15,6,True,2
498,2011-04-13,2011,4,13,2,False,2
346,2010-11-12,2010,11,12,4,False,4


## Analysis

### Churn Rate

In [61]:
"""
Monthly churn
"""

from datetime import timedelta

CHURN_THRESHOLD_DAYS = 120
churn_threshold = timedelta(days=CHURN_THRESHOLD_DAYS)

min_date = df_fct['purchase_timestamp'].min()
max_date = df_fct['purchase_timestamp'].max()
months_to_process = pd.date_range(min_date, max_date, freq='MS').normalize()
churn_results = []

for period_start in months_to_process:
    period_end = period_start + pd.offsets.MonthEnd(1)

    active_window_start = period_start - churn_threshold

    bop_transactions = df_fct[
        (df_fct['purchase_timestamp'] >= active_window_start) &
        (df_fct['purchase_timestamp'] < period_start)
    ]

    bop_customers = set(bop_transactions['customer_id'].unique())
    bop_customer_count = len(bop_customers)
    
    if bop_customer_count == 0:
        continue

    transactions_in_period = df_fct[
        (df_fct['purchase_timestamp'] >= period_start) &
        (df_fct['purchase_timestamp'] <= period_end)
    ]
    customers_who_purchased_in_period = set(transactions_in_period['customer_id'].unique())

    churned_customers = bop_customers - customers_who_purchased_in_period
    churned_customer_count = len(churned_customers)

    churn_rate = churned_customer_count / bop_customer_count if bop_customer_count > 0 else 0

    churn_results.append({
        'time_period': period_start.strftime('%Y-%m'),
        'bop_customer_count': bop_customer_count,
        'churned_customer_count': churned_customer_count,
        'churn_rate': round(churn_rate, 3)
    })

df_churn = pd.DataFrame(churn_results)
print(df_churn)

   time_period  bop_customer_count  churned_customer_count  churn_rate
0      2010-01                 955                     631       0.661
1      2010-02                1338                     961       0.718
2      2010-03                1714                    1117       0.652
3      2010-04                2139                    1510       0.706
4      2010-05                2163                    1494       0.691
5      2010-06                2289                    1600       0.699
6      2010-07                2404                    1717       0.714
7      2010-08                2321                    1676       0.722
8      2010-09                2303                    1564       0.679
9      2010-10                2391                    1537       0.643
10     2010-11                2747                    1714       0.624
11     2010-12                3110                    2375       0.764
12     2011-01                3026                    2445       0.808
13    

### RFM

In [88]:
"""
Recency:
- Define snapshot date (maximum purchase date + 1)
- Get latest purchase date for each customer ID
- Get diff between snapshot and latest purchase date

Frequency:
- Get the frequency of customer's transactions

Monetary:
- Get how much the customer spends

RFM Rank:
- Calculate each of the R, F, M rank
- Classify into 3 group since the total of unique customers < 30k
- Calculate the average of each of the R, F, M rank
- Round the final rank
"""

snapshot_date = df_fct['purchase_timestamp'].max() + timedelta(days=1)
df_rfm = (
    df_fct
    .assign(
        sales=lambda x: x['order_amt'] * x['product_price']
    )
    .groupby('customer_id', as_index=False)
    .agg(
        last_purchase_date=('purchase_timestamp', 'max'),
        # frequency
        frequency=('invoice_id', 'nunique'),
        # monetary
        monetary=('sales', 'sum')
    )
    .assign(
        # recency
        recency=lambda x: (snapshot_date - x['last_purchase_date']).dt.days,
        # R rank
        r_rank=lambda x: pd.qcut(x['recency'], q=3, labels=[3, 2, 1]),
        # F rank
        f_rank=lambda x: pd.qcut(x['frequency'], q=3, labels=[1, 2, 3]),
        # M rank
        m_rank=lambda x: pd.qcut(x['monetary'], q=3, labels=[1, 2, 3]),
        # RFM rank
        rfm_rank=lambda x: (
            x.loc[:, ['r_rank', 'f_rank', 'm_rank']]
            .mean(axis=1)
            .round()
            .astype('Int32')
        )
    )
    .loc[:, ['customer_id', 'r_rank', 'f_rank', 'm_rank', 'rfm_rank']]
)
display(df_rfm.head())

,customer_id,r_rank,f_rank,m_rank,rfm_rank
0,12346,1,3,3,2
1,12347,3,3,3,3
2,12348,2,2,3,2
3,12349,3,2,3,3
4,12350,1,1,1,1


### Churn + RFM

In [89]:
CHURN_THRESHOLD_DAYS = 120
churn_threshold = timedelta(days=CHURN_THRESHOLD_DAYS)

df_merged = (
    df_fct.merge(df_rfm, on='customer_id', how='left')
)

min_date = df_merged['purchase_timestamp'].min()
max_date = df_merged['purchase_timestamp'].max()
months_to_process = pd.date_range(min_date, max_date, freq='MS').normalize()
churn_results = []

for period_start in months_to_process:
    period_end = period_start + pd.offsets.MonthEnd(1)

    active_window_start = period_start - churn_threshold

    bop_transactions = df_merged[
        (df_merged['purchase_timestamp'] >= active_window_start) &
        (df_merged['purchase_timestamp'] < period_start)
    ]

    bop_customers = set(bop_transactions['customer_id'].unique())
    bop_customer_count = len(bop_customers)
    
    if bop_customer_count == 0:
        continue

    transactions_in_period = df_merged[
        (df_merged['purchase_timestamp'] >= period_start) &
        (df_merged['purchase_timestamp'] <= period_end)
    ]
    customers_who_purchased_in_period = set(transactions_in_period['customer_id'].unique())

    churned_customers = bop_customers - customers_who_purchased_in_period
    churned_customer_count = len(churned_customers)

    churn_rate = churned_customer_count / bop_customer_count if bop_customer_count > 0 else 0

    churn_results.append({
        'time_period': period_start.strftime('%Y-%m'),
        'bop_customer_count': bop_customer_count,
        'churned_customer_count': churned_customer_count,
        'churn_rate': round(churn_rate, 3)
    })

df_churn = pd.DataFrame(churn_results)
print(df_churn)

   time_period  bop_customer_count  churned_customer_count  churn_rate
0      2010-01                 955                     631       0.661
1      2010-02                1338                     961       0.718
2      2010-03                1714                    1117       0.652
3      2010-04                2139                    1510       0.706
4      2010-05                2163                    1494       0.691
5      2010-06                2289                    1600       0.699
6      2010-07                2404                    1717       0.714
7      2010-08                2321                    1676       0.722
8      2010-09                2303                    1564       0.679
9      2010-10                2391                    1537       0.643
10     2010-11                2747                    1714       0.624
11     2010-12                3110                    2375       0.764
12     2011-01                3026                    2445       0.808
13    